In [55]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
import tiktoken
import time

## OPCION 1 DE CARGAR LOS DOCUMENTOS

### No sirve muy bien esta opcion porque marca un error del "page_content", mejor usar la opcion 2

In [56]:
#loader1 = TextLoader("../raw_data/scraped_data.txt")
#text1 = loader1.load()

loader2 = TextLoader("../raw_data/scraped_data2.txt")
text2 = loader2.load()

#loader3 = PyPDFLoader("../raw_data/Reglamento_CDMX.pdf")
#pdf = loader3.load()


In [57]:
documents = [text2]

## OPCION 2 DE CARGAR LOS DOCUMENTOS

### Este cicllo se hizo solo con el SEGUNDO DOCUMENTO TXT 
Tiene 247 docs y 84591 tokens

In [58]:
file_paths = [
    #../raw_data/scraped_data.txt",
    "../raw_data/scraped_data2.txt",
    #"../raw_data/Reglamento_CDMX.pdf"
]

In [59]:
documents = []
for file_path in file_paths:
    if file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    elif file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    else:
        print(f"Unsupported file format for {file_path}")
text = loader.load()
documents.append(text)

In [60]:
print(len(documents))

1


In [61]:
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)


In [62]:
docs = text_splitter.split_documents(documents[0])

Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500
Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500
Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500


In [63]:
print(len(docs))

231


In [64]:
def countTokens(query: str):
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens=len(encoding.encode(query))
    return num_tokens

In [65]:
print(sum(countTokens(d.page_content) for d in docs))

84591


### Aqui hacemos el embedding solo a una parte de los 247 docs del segundo txt, para probar unicamente

In [66]:
i=1
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents([docs[0]],embeddings)
while i <= 50:
    db1 = FAISS.from_documents(docs[i:i+20], embeddings)
    i +=20
    db.merge_from(db1)
    print(i)
    time.sleep(30)


21
41
61


In [67]:
query = "Cuanto cuesta renovar la licencia"
docs = db.similarity_search(query)

In [68]:
print(docs[0].page_content)

1. Antes de iniciar tu trámite deberás leer cuidadosamente los requisitos a reunir. Como primer paso deberás realizar el pago de derechos por concepto de "Renovación o Refrendo de tarjeta de circulación",  Clave 36-15  Descárgala aquí: https://data.finanzas.cdmx.gob.mx/formato_lc/vehicular/36_15.  Realiza tu pago en los Centros de Servicio de la Secretaría de Administración y Finanzas o Auxiliares de pago autorizados, mismos que podrás consultar a través de la siguiente liga https://s3.amazonaws.com/cdmxassets/media/Archivos+pdf/Directorio_areas_web.pdf, considera que en algunos establecimientos la aplicación de tu pago se verá reflejado hasta 72 horas después. Conserva el comprobante de pago de la línea de captura es parte de la documentación que tendrás que presentar.


2. Una vez que tengas los requisitos completos además del comprobante de pago en original, solicita una cita en SEMOVI en alguno de los Módulos de Control Vehicular disponibles en la siguiente liga: https://app.semovi

In [69]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
print(docs[0].page_content)

1. Antes de iniciar tu trámite deberás leer cuidadosamente los requisitos a reunir. Como primer paso deberás realizar el pago de derechos por concepto de "Renovación o Refrendo de tarjeta de circulación",  Clave 36-15  Descárgala aquí: https://data.finanzas.cdmx.gob.mx/formato_lc/vehicular/36_15.  Realiza tu pago en los Centros de Servicio de la Secretaría de Administración y Finanzas o Auxiliares de pago autorizados, mismos que podrás consultar a través de la siguiente liga https://s3.amazonaws.com/cdmxassets/media/Archivos+pdf/Directorio_areas_web.pdf, considera que en algunos establecimientos la aplicación de tu pago se verá reflejado hasta 72 horas después. Conserva el comprobante de pago de la línea de captura es parte de la documentación que tendrás que presentar.


2. Una vez que tengas los requisitos completos además del comprobante de pago en original, solicita una cita en SEMOVI en alguno de los Módulos de Control Vehicular disponibles en la siguiente liga: https://app.semovi

In [70]:
db.save_local("faiss_prueba")

In [71]:
new_db = FAISS.load_local("faiss_prueba", embeddings)

In [72]:
docs = new_db.similarity_search(query)
docs[0]

Document(page_content='1. Antes de iniciar tu trámite deberás leer cuidadosamente los requisitos a reunir. Como primer paso deberás realizar el pago de derechos por concepto de "Renovación o Refrendo de tarjeta de circulación",  Clave 36-15  Descárgala aquí: https://data.finanzas.cdmx.gob.mx/formato_lc/vehicular/36_15.  Realiza tu pago en los Centros de Servicio de la Secretaría de Administración y Finanzas o Auxiliares de pago autorizados, mismos que podrás consultar a través de la siguiente liga https://s3.amazonaws.com/cdmxassets/media/Archivos+pdf/Directorio_areas_web.pdf, considera que en algunos establecimientos la aplicación de tu pago se verá reflejado hasta 72 horas después. Conserva el comprobante de pago de la línea de captura es parte de la documentación que tendrás que presentar.\n\n\n2. Una vez que tengas los requisitos completos además del comprobante de pago en original, solicita una cita en SEMOVI en alguno de los Módulos de Control Vehicular disponibles en la siguient

## PRUEBA CARGANDO SOLO EL PDF

### El pdf tiene 90 docs y 52884 tokens

In [73]:
file_paths = [
    #../raw_data/scraped_data.txt",
    #"../raw_data/scraped_data2.txt",
    "../raw_data/Reglamento_CDMX.pdf"
]

In [74]:
documents = []
for file_path in file_paths:
    if file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    elif file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    else:
        print(f"Unsupported file format for {file_path}")
text = loader.load()
documents.append(text)

In [75]:
print(len(documents))

1


In [76]:
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)

In [77]:
docs = text_splitter.split_documents(documents[0])

In [78]:
print(len(docs))

90


In [79]:
def countTokens(query: str):
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens=len(encoding.encode(query))
    return num_tokens

print(sum(countTokens(d.page_content) for d in docs))

52884


### Aqui hacemos el embedding solo a una parte de los 90 docs del pdf, para probar unicamente

In [80]:
i=1
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents([docs[0]],embeddings)
while i <= 50:
    db1 = FAISS.from_documents(docs[i:i+20], embeddings)
    i +=20
    db.merge_from(db1)
    print(i)
    time.sleep(30)

21
41
61


In [81]:
query = "Cuanto cuesta renovar la licencia"
docs = db.similarity_search(query)
print(docs[0].page_content)

conductores de vehículos motorizados que infrinjan la presente disposición serán sancionados con una multa equivalente a 5, 7 o 10 veces la Unidad de Medida y Actualización vigente y tres puntos de penalización a la licencia para conducir o en caso de infracciones captadas a través de sistemas tecnológicos se sancionará con un punto a la matrícula vehicular.


In [82]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
print(docs[0].page_content)

conductores de vehículos motorizados que infrinjan la presente disposición serán sancionados con una multa equivalente a 5, 7 o 10 veces la Unidad de Medida y Actualización vigente y tres puntos de penalización a la licencia para conducir o en caso de infracciones captadas a través de sistemas tecnológicos se sancionará con un punto a la matrícula vehicular.


In [83]:
db.save_local("faiss_prueba_pdf")

In [84]:
new_db = FAISS.load_local("faiss_prueba_pdf", embeddings)

In [85]:
docs = new_db.similarity_search(query)
docs[0]

Document(page_content='conductores de vehículos motorizados que infrinjan la presente disposición serán sancionados con una multa equivalente a 5, 7 o 10 veces la Unidad de Medida y Actualización vigente y tres puntos de penalización a la licencia para conducir o en caso de infracciones captadas a través de sistemas tecnológicos se sancionará con un punto a la matrícula vehicular.', metadata={'source': '../raw_data/Reglamento_CDMX.pdf', 'page': 13})

## PRUEBA CARGANDO EL SEGUNDO TXT Y EL PDF

### El pdf tiene 90 docs y 52884 tokens
### El segundo txt dijimos que tiene 247 docs y 84591 tokens

In [102]:
file_paths = [
    #../raw_data/scraped_data.txt",
    "../raw_data/scraped_data2.txt",
    "../raw_data/Reglamento_CDMX.pdf"
]

Con este for no se podian cargar los dos documentos, el txt y el pdf

In [87]:
documents = []
for file_path in file_paths:
    if file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    elif file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    else:
        print(f"Unsupported file format for {file_path}")
text = loader.load()
documents.append(text)

Se cargaron de esta manera:

In [107]:
documents =[]

loader2 = TextLoader("../raw_data/scraped_data2.txt")
text2 = loader2.load()
documents.append(text2)

loader3 = PyPDFLoader("../raw_data/Reglamento_CDMX.pdf")
pdf = loader3.load()
documents.append(pdf)

In [114]:
print(len(documents))

2


In [109]:
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)

In [112]:
docs_0 = text_splitter.split_documents(documents[0])
print(len(docs_0))
print(type(docs_0))

Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500
Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500
Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500


231
<class 'list'>


In [115]:
docs_1 = text_splitter.split_documents(documents[1])
print(len(docs_1))
print(type(docs_1))

90
<class 'list'>


In [125]:
docs = []

for doc in documents:
    split_doc = text_splitter.split_documents(doc)
    docs.append(split_doc)


Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500
Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500
Created a chunk of size 2172, which is longer than the specified 1500
Created a chunk of size 1843, which is longer than the specified 1500
Created a chunk of size 4670, which is longer than the specified 1500
Created a chunk of size 4138, which is longer than the specified 1500


In [124]:
len(docs)

2

In [122]:
print(f"Primer doc tiene {len(docs[0])} documentos y Segundo doc tiene {len(docs[1])}")

Primer doc tiene 231 documentos y Segundo doc tiene 90


### Aqui hacemos el embedding solo a una parte de los 337 docs que suman del pdf y del segundo txt, para probar unicamente

In [139]:
embeddings = OpenAIEmbeddings(show_progress_bar = True)
db = FAISS.from_documents([docs[0][0]],embeddings)
for doc in docs:
    i=1
    print(f"Creating embedding from document...")
    while i <= 50:
        db1 = FAISS.from_documents(doc[i:i+20], embeddings)
        i +=20
        db.merge_from(db1)
        print(i)
        time.sleep(30)

/home/tere/.pyenv/versions/3.10.6/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.99it/s]


Creating embedding from document...


100%|██████████| 1/1 [00:00<00:00,  1.42it/s]


21


...

21


100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


41


100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


61


In [140]:
query = "Cuanto cuesta renovar la licencia"
docs = db.similarity_search(query)
print(docs[0].page_content)

100%|██████████| 1/1 [00:00<00:00,  3.25it/s]

1. Antes de iniciar tu trámite deberás leer cuidadosamente los requisitos a reunir. Como primer paso deberás realizar el pago de derechos por concepto de "Renovación o Refrendo de tarjeta de circulación",  Clave 36-15  Descárgala aquí: https://data.finanzas.cdmx.gob.mx/formato_lc/vehicular/36_15.  Realiza tu pago en los Centros de Servicio de la Secretaría de Administración y Finanzas o Auxiliares de pago autorizados, mismos que podrás consultar a través de la siguiente liga https://s3.amazonaws.com/cdmxassets/media/Archivos+pdf/Directorio_areas_web.pdf, considera que en algunos establecimientos la aplicación de tu pago se verá reflejado hasta 72 horas después. Conserva el comprobante de pago de la línea de captura es parte de la documentación que tendrás que presentar.


2. Una vez que tengas los requisitos completos además del comprobante de pago en original, solicita una cita en SEMOVI en alguno de los Módulos de Control Vehicular disponibles en la siguiente liga: https://app.semovi

In [143]:
query = "Cual es la multa por manejar sin licencia"
docs = db.similarity_search(query)
print(docs[0].page_content)

100%|██████████| 1/1 [00:00<00:00,  2.98it/s]

conductores de vehículos motorizados que infrinjan la presente disposición serán sancionados con una multa equivalente a 5, 7 o 10 veces la Unidad de Medida y Actualización vigente y tres puntos de penalización a la licencia para conducir o en caso de infracciones captadas a través de sistemas tecnológicos se sancionará con un punto a la matrícula vehicular.


In [144]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
print(docs[0].page_content)

100%|██████████| 1/1 [00:00<00:00,  3.04it/s]

conductores de vehículos motorizados que infrinjan la presente disposición serán sancionados con una multa equivalente a 5, 7 o 10 veces la Unidad de Medida y Actualización vigente y tres puntos de penalización a la licencia para conducir o en caso de infracciones captadas a través de sistemas tecnológicos se sancionará con un punto a la matrícula vehicular.


In [151]:
db.save_local("faiss_prueba_pdf_txt")

In [146]:
new_db = FAISS.load_local("faiss_prueba_pdf_txt", embeddings)

In [147]:
docs = new_db.similarity_search(query)
docs[0]

100%|██████████| 1/1 [00:00<00:00,  3.13it/s]


Document(page_content='conductores de vehículos motorizados que infrinjan la presente disposición serán sancionados con una multa equivalente a 5, 7 o 10 veces la Unidad de Medida y Actualización vigente y tres puntos de penalización a la licencia para conducir o en caso de infracciones captadas a través de sistemas tecnológicos se sancionará con un punto a la matrícula vehicular.', metadata={'source': '../raw_data/Reglamento_CDMX.pdf', 'page': 13})

In [150]:
docs[3].metadata

{'source': '../raw_data/Reglamento_CDMX.pdf', 'page': 37}